<a href="https://colab.research.google.com/github/ariesabdillah/homework_spark/blob/main/spark_latihan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Reqruiment

In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

#unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

#set your spark folder to your system path environment
import os
os.environ["JAVA_HOME"] = "usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

#install findspark using pip
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 15.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=73ac767e6f45b517d368d1c39ff5d5af0a833ad01c481959127420ee6627025e
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
#set your spark folder to your system path environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

## make Spark session

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

## Import Data from google drive

In [ ]:
df = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("/content/gdrive/MyDrive/fhv_tripdata_2019-01.csv.gz")
print("Here is our inferred schema:")
df.printSchema()

Here is our inferred schema:
root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [ ]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00001|2019-01-01 00:30:00|2019-01-01 02:51:55|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:45:00|2019-01-01 00:54:49|        null|        null|   null|                B00001|
|              B00001|2019-01-01 00:15:00|2019-01-01 00:54:52|        null|        null|   null|                B00001|
|              B00008|2019-01-01 00:19:00|2019-01-01 00:39:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:27:00|2019-01-01 00:37:00|        null|        null|   null|                B00008|
|              B00008|2019-01-01 00:48:0

In [ ]:
df.createOrReplaceTempView("tripData")

## Transform data pickup_datetime mulai tanggal 1-10 januari dan PUlocationID dan DOlocationID not null

In [ ]:
sqlTripData = spark.sql("SELECT PUlocationID, DOlocationID, pickup_datetime, dropoff_datetime \
FROM tripData WHERE PULocationID IS NOT NULL and DOLocationID is NOT NULL AND pickup_datetime >= '2019-01-01' \
AND pickup_datetime <='2019-01-10' GROUP BY pickup_datetime, DOlocationID, PUlocationID, dropoff_datetime \
ORDER BY pickup_datetime")

In [ ]:
sqlTripData.show()

+------------+------------+-------------------+-------------------+
|PUlocationID|DOlocationID|    pickup_datetime|   dropoff_datetime|
+------------+------------+-------------------+-------------------+
|         231|         234|2019-01-01 00:00:01|2019-01-01 00:29:55|
|         229|         188|2019-01-01 00:00:01|2019-01-01 00:32:57|
|          81|         250|2019-01-01 00:00:01|2019-01-01 00:13:55|
|          28|         131|2019-01-01 00:00:01|2019-01-01 00:15:42|
|         160|          36|2019-01-01 00:00:01|2019-01-01 00:10:22|
|          69|          60|2019-01-01 00:00:02|2019-01-01 00:12:03|
|         216|         216|2019-01-01 00:00:02|2019-01-01 00:04:54|
|          71|          72|2019-01-01 00:00:02|2019-01-01 00:09:53|
|          77|         188|2019-01-01 00:00:02|2019-01-01 00:24:12|
|         137|         114|2019-01-01 00:00:02|2019-01-01 00:11:30|
|          61|          62|2019-01-01 00:00:02|2019-01-01 00:08:10|
|          86|         122|2019-01-01 00:00:02|2

## Write data parquet

In [ ]:
sqlTripData.write.parquet("TripData_parquet.parquet")


## Write data json

In [ ]:
sqlTripData.write.json("TripData_json.json")